In [70]:
# import libraries
#import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import gzip
import pickle
import os
from urllib.request import urlretrieve
import numpy as np
from matplotlib import image


def conv(image, params, s = 1, pad = 1 ): # s = 1 (conv stride)
    #f1 shape : (num_filters,input channels, f_h, f_w)
    #image shape: (channels, height, width)
    [f, b] = params
    if(pad >= 1):
        tmp = np.zeros((image.shape[0],image.shape[1]+2*pad ,image.shape[2]+2*pad))
        tmp[:,pad:-pad,pad:-pad] = image
        image = tmp    
        
    f_num = f.shape[0]
    f_depth = f.shape[1] #f1 = 1 , f2 = 8
    f_size = f.shape[2] 
        

    h_range = int((image.shape[1] - f_size) / s) + 1     # (W - F + 2P) / S  
    w_range = int((image.shape[2] - f_size) / s) + 1     # (W - F + 2P) / S  
    np_o = np.zeros((f_num, h_range, w_range))
    for z in range(f_num): # Number of filters
        for _h in range(h_range):      
            for _w in range(w_range):
                np_o[z, _h, _w] = np.sum(np.multiply(image[:, _h*s : _h*s + f_size, _w*s : _w*s + f_size] , f[z, :, :, :])) + b[z]
    
    return np_o
def conv_alg(image, params, s = 1, pad = 1 ): # s = 1 (conv stride)
    #f1 shape : (num_filters,input channels, f_h, f_w)
    #image shape: (channels, height, width)
    [f, b] = params
    if(pad >= 1):
        tmp = np.zeros((image.shape[0],image.shape[1]+2*pad ,image.shape[2]+2*pad))
        tmp[:,pad:-pad,pad:-pad] = image
        image = tmp    
        
    f_num = f.shape[0]
    f_size = f.shape[2] 
    
    np_o_t = np.zeros((f_num, image.shape[1] + (f_size-1), image.shape[2]+ (f_size-1)))
    for z in range(f_num): # Number of filters
        for _h in range(image.shape[1]):      
            for _w in range(image.shape[2]):
                np_o_t[z,_h:(_h+f_size),_w:(_w+f_size)] += np.sum((image[:, _h, _w].reshape(-1,1,1)*f[z, :, :, :]),axis=0)
        np_o_t[z,:,:] += b[z]*np.ones((image.shape[1] + (f_size-1),image.shape[2] + (f_size-1)))
    return np_o_t[:,(f_size-1):-(f_size-1), (f_size-1):-(f_size-1)]

In [94]:
x = np.ones((1,7,7))

x[:,0,1:4]=1
x[:,1,2:5]=2
x[:,2,3:6]=3
x[:,3,3:5]=4
x[:,4,2:4]=5
x[:,5,1:3]=6
x[:,6,0:2]=7

#k = np.random.rand(1,2,3,3)
k=np.ones((1,1,3,3))
k[:,:,0,1]=1
k[:,:,1,0]=2
k[:,:,1,2]=3
k[:,:,2,1]=4


b=np.zeros((1))

In [95]:
print(x)
print(k)

[[[1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 2. 2. 2. 1. 1.]
  [1. 1. 1. 3. 3. 3. 1.]
  [1. 1. 1. 4. 4. 1. 1.]
  [1. 1. 5. 5. 1. 1. 1.]
  [1. 6. 6. 1. 1. 1. 1.]
  [7. 7. 1. 1. 1. 1. 1.]]]
[[[[1. 1. 1.]
   [2. 1. 3.]
   [1. 4. 1.]]]]


In [96]:
params = [k,b]
res_ref = conv(x,params,1,0)#padding = 0
res = conv_alg(x,params,1,0)
print(res_ref)
print(res)

[[[18. 21. 31. 30. 27.]
  [16. 26. 41. 44. 25.]
  [19. 46. 51. 34. 25.]
  [52. 59. 38. 29. 18.]
  [69. 44. 33. 19. 15.]]]
[[[17. 20. 25. 25. 22.]
  [16. 27. 33. 38. 27.]
  [19. 31. 42. 43. 34.]
  [33. 40. 51. 42. 18.]
  [46. 61. 50. 19. 15.]]]


In [93]:
print(res-res_ref)

[[[ -2.  -2. -12. -10. -10.]
  [  0.   2. -16. -12.   4.]
  [  0. -30. -18.  18.  18.]
  [-38. -38.  26.  26.   0.]
  [-46.  34.  34.   0.   0.]]]
